##### an attemp to change the code so that is uses new standard format but 1 ambiguity: 2 gdsc ids map to same standard drug name..which ic50/curve catgeory to chose?

### Script to
1. Get CCLE and GDSC curve categories  from previous task file and assign DW (2) label to pairs for which ic50<max_dosage 
so that we can Check if Hump and rising curves are outliers (to be removed) or is it the property of some dr/cl ? 

(if it is the prop, then same pairs would be Hump in both datasets/ Rising in both datasets)
###### **NOTE: this script used dose response files in the old format : cl,dr have dataset specific names, 1 row for 1 pair not 1 doasge

In [1]:
__author__ = 'Aanchal'

In [2]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import FUNCTIONS as ff

In [3]:
import warnings
warnings.filterwarnings('ignore')


In [43]:
# define these 4 parameters based on which method you want to use to separate normal shape category 1) IC50 2) code
ccle_max_dose=3 #log2(8)
categoryMatrix_ccle=pd.read_csv("../data_processed/CurveCategoryDetection/"+"CCLE"+"_CurveCategory.tsv", sep='\t', index_col=0)
categoryMatrix_gdsc=pd.read_csv("../data_processed/CurveCategoryDetection/"+"GDSE"+"_CurveCategory.tsv", sep='\t', index_col=0)

[dic_cl_ccle,dic_dr_ccle]=ff.getDictionary2('CCLE') #metadata_filename="../data/metadata_2019_08_15_aanchal.xlsx"#meta_df = pd.read_excel(metadata_filename,sheetname="cell line")#cl_name_dict = dict(zip(meta_df['CCLE name'], meta_df['name']))#ccle.columns = [cl_name_dict[cl] for cl in ccle.columns]#[cl for cl in ccle.columns if cl not in cl_name_dict.keys()]
categoryMatrix_ccle=categoryMatrix_ccle.rename(index=dic_cl_ccle) #skips renaming cls which are not present in dictionary
categoryMatrix_ccle=categoryMatrix_ccle.rename(columns=dic_dr_ccle)

[dic_cl_gdsc,dic_dr_gdsc]=ff.getDictionary2('GDSC')
categoryMatrix_gdsc.index = categoryMatrix_gdsc.index.map(str)
categoryMatrix_gdsc.columns = categoryMatrix_gdsc.columns.map(int)
categoryMatrix_gdsc=categoryMatrix_gdsc.rename(index=dic_cl_gdsc)
categoryMatrix_gdsc=categoryMatrix_gdsc.rename(columns=dic_dr_gdsc)
#For selumetnib and PLX-720,, 2 gdsc ids would map to same standard names

In [54]:
categoryMatrix_gdsc.loc['COLO-829','Rapamycin']

4.0

In [56]:
ic50_ccle.loc['MEG-01','Nilotinib']

-3.8390660538699994

##### Get (dataset specific) names of common pairs

In [46]:
datasets_list=['CCLE','GDSC']
rawFileDir="../data/"
[Intersection,  Intersection_names] = ff.getCommonPairs(datasets_list, rawFileDir)

0
1


In [47]:
### splitting normal shape category using IC50
[ic50_ccle, m_ccle]=ff.get_ic50slope('CCLE')
[ic50_gdsc,m_gdsc]=ff.get_ic50slope('GDSC')

In [77]:
for i, row in Intersection_names.iterrows():
    print(i)
    drname= row['Standard drug name']
    clname= row['Standard cell line name']
    print(clname, drname)
   
    ccle_label= categoryMatrix_ccle.loc[clname , drname  ]
    Intersection_names.loc[i,'ccle_category'] =  ccle_label
    
    if (ccle_label==1): ##not in prev code
        ccle_ic50= ic50_ccle.loc[clname , drname  ]
        if (ccle_ic50 < ccle_max_dose):
            Intersection_names.loc[i,'ccle_category'] = 2
            

            
    gdsc_label= categoryMatrix_gdsc.loc[clname , drname  ]
    print(gdsc_label)
    print('*')
    Intersection_names.loc[i,'gdsc_category'] =  gdsc_label
    
    if (gdsc_label==1):
        
        t1 = Intersection[ Intersection['Standard cell line name']==clname] 
        t2 = t1[t1['Standard drug name']==drname]
        gdsc_max_dose= t2['Doses (uM)'].max()
        
        gdsc_ic50= ic50_gdsc.loc[clname , drname ]

        if (gdsc_ic50 < gdsc_max_dose):
            Intersection_names.loc[i,'gdsc_category'] = 2

0
22RV1 Tanespimycin
1.0
*
1
22RV1 PLX-4720
PLX-4720    1.0
PLX-4720    1.0
Name: 22RV1, dtype: float64
*


ValueError: Incompatible indexer with Series

In [78]:
ic50_gdsc.to_csv('s2.csv')

In [83]:
Intersection

,Standard cell line name,Standard drug name,Doses (uM),Response,dataset,ccle_category
0,22RV1,Nilotinib,0.002500,9.980000,CCLE,2.0
1,22RV1,Nilotinib,0.008000,7.660000,CCLE,1.0
2,22RV1,Nilotinib,0.025000,-2.200000,CCLE,1.0
3,22RV1,Nilotinib,0.080000,15.100000,CCLE,1.0
4,22RV1,Nilotinib,0.250000,29.500000,CCLE,2.0
5,22RV1,Nilotinib,0.800000,21.200000,CCLE,2.0
6,22RV1,Nilotinib,2.530000,12.300000,CCLE,2.0
7,22RV1,Nilotinib,8.000000,-14.000000,CCLE,1.0
8,22RV1,Nilotinib,2.000000,15.971455,GDSC,1.0
9,22RV1,Nilotinib,1.000000,-7.156625,GDSC,2.0


In [7]:
Intersection.to_csv('../data_processed/CurveCategoryDetection/Intersection_withCurveCategory_SplitNormalUsingIC50.csv',index=False)
#Intersection_ic50=pd.read_csv('../data_processed/CurveCategoryDetection/Intersection_withCurveCategory_SplitNormalUsingIC50.csv')


In [ ]:
Intersection_ic50=Intersection

In [12]:
ic50_gdsc

,Unnamed: 0,1024,1025,1026,3,1028,1029,1030,1031,1032,...,1526,1527,1016,1529,1018,1019,1020,170,1022,1023
0,1240121,-0.118994,NaN,-3.907545,NaN,4.382521,9.272281,5.432403,-7.196907,-0.883702,...,0.706309,0.705568,-6.007790,-1.064632,7.342955,-0.137528,4.859275,1.426957,-0.742743,3.144137
1,1240122,-3.528531,7.307202,-3.223007,NaN,9.584129,5.998849,3.607405,-7.030159,-5.409668,...,1.124453,0.714962,0.046558,-0.898379,4.822124,-2.323476,6.476351,-0.275244,-2.838419,1.430701
2,1240123,1.376514,5.259768,0.163071,NaN,5.804456,7.640829,6.710514,-1.162185,-1.574530,...,6.492452,3.189233,6.214937,7.714992,7.880840,1.310319,5.252832,2.350767,-0.367406,7.291675
3,1240124,2.329470,10.575640,-2.382794,NaN,11.213035,11.189297,14.795448,11.437178,0.099986,...,1.230434,1.095624,11.202682,9.733274,9.936282,4.926269,11.112255,NaN,1.273545,12.331214
4,1240125,1.429638,NaN,1.444760,NaN,11.546682,0.759840,4.904118,0.791628,10.847021,...,1.792962,3.235314,2.623841,-0.117098,9.567237,10.419022,9.966728,3.245973,-1.846780,1.434749
5,1240127,2.522769,10.709400,0.345885,NaN,9.811162,8.698021,4.682884,-5.386643,8.868653,...,NaN,2.725299,4.624977,0.652045,10.308761,8.996660,10.429243,4.491150,-1.735400,10.080419
6,1240128,-2.502069,7.868777,-0.664026,3.631274,6.066793,6.199072,7.497937,-7.673491,5.098507,...,2.060082,7.559630,2.233110,-0.115861,6.119705,-0.457301,7.522441,0.797840,-3.737170,5.945542
7,1240129,1.861973,9.085553,0.375961,NaN,8.965243,7.876012,4.680783,-4.964602,4.567243,...,3.534286,1.898644,3.688571,7.428490,9.006907,6.086904,8.694509,-0.465719,0.245362,7.063826
8,1240130,-4.055747,4.715653,NaN,NaN,3.245119,NaN,NaN,-8.110191,5.375921,...,-0.943431,4.432749,-6.081473,-1.293673,8.224272,-0.153823,9.443382,-1.059258,-3.255498,2.481057
9,1240131,NaN,NaN,0.072241,NaN,NaN,7.932834,3.735736,NaN,NaN,...,4.911254,2.101175,NaN,-0.963735,NaN,NaN,NaN,0.452908,NaN,NaN
